## Marc Notebook

## 1) import usefuls module

In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import math 

from itertools import groupby
from helpers import *

from opt_helpers_marc import *
from plots import *
import scipy
import scipy.io
import numpy as np
import scipy.sparse as sp

%load_ext autoreload
%autoreload 2

## 2) Load the data

In [2]:
# write here the path to the dataset
path_dataset = "datasets/data_train.csv"

# load the data 
ratings = load_data(path_dataset)
ratings = ratings.todense()

number of items: 10000, number of users: 1000


## 3) Split the data 

In [3]:
# fraction of the data set that will be the test set 
p_test = 0.5

train, test = opt_split_data(ratings, p_test)
print("data split done")

data split done


## First test of matrix factorization : overflow ? 

In [13]:
K = 10
num_epochs = 40
lambda_user = 1e-7
lambda_item = 1e-7
gamma = 0.1

items, users, rmse_test = opt_matrix_factorization_SGD(train, test, K, num_epochs, lambda_user, lambda_item, gamma)

initialization of the factorization : done
learn the matrix factorization using SGD...
iter: 0, RMSE on training set: 1.76396297262.
iter: 1, RMSE on training set: 1.64196096825.
iter: 2, RMSE on training set: 1.56605146636.
iter: 3, RMSE on training set: 1.52577419967.
iter: 4, RMSE on training set: 1.48270209707.
iter: 5, RMSE on training set: 1.45362480025.
iter: 6, RMSE on training set: 1.43931128142.
iter: 7, RMSE on training set: 1.42130140107.
iter: 8, RMSE on training set: 1.41263053146.
iter: 9, RMSE on training set: 1.39873461433.
iter: 10, RMSE on training set: 1.39329548055.
iter: 11, RMSE on training set: 1.38688689686.
iter: 12, RMSE on training set: 1.38239702643.
iter: 13, RMSE on training set: 1.37851341262.
iter: 14, RMSE on training set: 1.37530305555.
iter: 15, RMSE on training set: 1.37291351833.
iter: 16, RMSE on training set: 1.37099876241.
iter: 17, RMSE on training set: 1.36987847865.
iter: 18, RMSE on training set: 1.36852726726.
iter: 19, RMSE on training set

## 4) Matrix Factorization through Stochastic Gradient Descent 

In [ ]:
## Perform matrix factorization through stochastic gradient descent, for different numbers of features
# (grid search on the number of feature)
# keep the best number of feature, and the associated factorization

## Parameters for the matrix factorization
# maximum number of features fixed
max_K = 15

# incrementation step for K
step_K = 2

# number of full iterations of the stochastic gradient descent
max_epochs = 10

# regularization parameters (regularization of the loss function in Gradient Descent)
lambda_user = 1e-3
lambda_item = 1e-3

# descent step size
gamma = 0.1

# initialization of variables refering to the matrix factorization for the best number
best_rmse_test_sgd = float('Inf')
best_k_sgd = 0
best_user_feat_sgd = np.zeros(1)
best_item_feat_sgd = np.zeros(1)

# for each K of the grid, compute the matrix factorization (training set) and the rmse (test set), 
# If it improves the results, update the rmse

for K in range(1,max_K,step_K):
    
    print("matrix factorization for the number of features : ", K)
    
    # compute the stochastic gradient descent matrix factorization
    user_features, item_features, rmse_test = matrix_factorization_SGD(train, test, K, max_epochs, lambda_user, lambda_item, gamma)
    if rmse_test < best_rmse_test_sgd:
        # better rmse => update the references
        best_rmse_test_sgd = rmse_test
        best_k_sgd = K
        best_user_feat_sgd = user_features
        best_item_feat_sgd = item_features

# print the results
print("best number of features found : ", best_k_sgd)
print("rmse on the test set for this number of features : ", best_rmse_test_sgd)


## 5) Matrix Factorization Using alternative Least Square Method 

In [ ]:
## Perform matrix factorization through stochastic gradient descent, for different numbers of features
# (grid search on the number of feature)
# keep the best number of feature, and the associated factorization

## Parameters for the matrix factorization
# maximum number of features fixed
max_K = 6

# incrementation step for K
step_K = 1

# regularization parameters for the loss function 
lambda_user = 1e-8
lambda_item = 1e-8

# stop criterion
stop_criterion = 1e-3

# number of iterations of the alternative least squares method 
it_max = 10

# initialization of variables refering to the matrix factorization for the best number
best_rmse_test_als = float('Inf')
best_k_als = 0
best_user_feat_als = np.zeros(1)
best_item_feat_als = np.zeros(1)

for K in range(1,max_K,step_K):
    
    print("als matrix factorization for the number of features : ", K)
    
    # compute the matrix factorization using alternative least squares method
    user_features, item_features, rmse_test =ALS(train, test,K, lambda_user, lambda_item, stop_criterion, it_max)
    print("rmse on test data for ", K, " features : ", rmse_test)
    if rmse_test < best_rmse_test_als:
        # better rmse => update the references
        best_rmse_test_als = rmse_test
        best_k_als = K
        best_user_feat_als = user_features
        best_item_feat_als = item_features
        
# print the results
print("best number of features found : ", best_k_als)
print("rmse on the test set for this number of features : ", best_rmse_test_als)

## 6) Prediction using SGD Matrix Factorization

In [ ]:
### Prediction using the SGD Matrix Factorization best parameters found previously 

# load the submission data set 
# write here the path to the dataset
path_dataset_submit = "../datasets/sampleSubmission.csv"

# load the data 
ratings_submit = load_data(path_dataset_submit)
num_items_per_user, num_users_per_item = plot_raw_data(ratings_submit)

#ratings_submit = ratings_submit[:30,:50]
print("ok")

In [ ]:
# Need to apply the same transformation and keep only in the training set the items,users for which there are at least
# a certain amount of information => train_submit : FALSE => UNCOMPATIBLE SHAPE
#min_num_ratings = 10
#dummy_valid_ratings_submit, train_submit, dummy_test_submit = split_data(ratings_submit, num_items_per_user, num_users_per_item, min_num_ratings, 0)
print("ok")

In [ ]:
# SAME PARAMETERS AS DURING THE TRAINING
# number of full iterations of the stochastic gradient descent
max_epochs =20

# regularization parameters (regularization of the loss function in Gradient Descent)
lambda_user = 1e-3
lambda_item = 1e-7

# descent step size
gamma = 0.1
#ratings_submit = ratings_submit[:1000,:1000]
# size of the rating matrix
d,n = ratings_submit.shape
print("d,n :", d,n)
# for test only 
best_k_sgd = 5

# compute the prediction matrix 
# => directly use the ratings matrix as rating as training set 
user_features_submit_sgd, item_features_submit_sgd, rmse_full_ratings = matrix_factorization_SGD(ratings_submit, ratings_submit, best_k_sgd, max_epochs, lambda_user, lambda_item, gamma)

# print : rmse of our full prediction : 
print("rmse of our prediction : ", rmse_full_ratings)